In [5]:
import pandas as pd
import pyarrow.dataset as pqds
from pyarrow import fs
import pyarrow as pa

In [3]:
data_path = "data/kraken.ETHUSD"
data_set_index = "dsindex"
begin_partition = "202201"
end_partition = "202212"
# Read parquet file from data_path as pyarrown dateset
dataset: pqds.dataset = pqds.dataset(
            data_path,
            format="parquet",
            partitioning="hive",
            filesystem=fs.LocalFileSystem(),
        )

criteria = (pqds.field(data_set_index) >= begin_partition) & (
            pqds.field(data_set_index) <= end_partition
        )
table: pa.table = dataset.to_table(filter=criteria)
